## ECC TESTBENCH

This notebook takes one input (Data) and gives the corresponding ECC bits using AXI-STREAM. It is then compared with precomputed expected values

In [15]:
from pynq import Overlay
import numpy as np
from pynq import Xlnk
from pynq.lib import dma
from scipy.linalg import dft
import matplotlib.pyplot as plt

In [16]:
ol=Overlay('design_1.bit')

In [17]:
dma1=ol.axi_dma_0


In [18]:
NUM_SAMPLES = 256

real_error=np.zeros(NUM_SAMPLES)
imag_error=np.zeros(NUM_SAMPLES)
ind=np.arange(NUM_SAMPLES)
real_rmse=np.zeros(NUM_SAMPLES)
imag_rmse=np.zeros(NUM_SAMPLES)

In [20]:
xlnk = Xlnk()
in_r = xlnk.cma_array(shape=(NUM_SAMPLES,), dtype=np.int8) 
out_r = xlnk.cma_array(shape=(NUM_SAMPLES,), dtype=np.int8) 


In [21]:


img1=np.random.randint(255,size=256)

for i in range(256): 
    img1[i] = i

np.copyto(in_r, img1)


In [22]:
length = NUM_SAMPLES
dft_ip = ol.ecc_encoder_0
dft_ip.write(0x00,1)
dma1.recvchannel.start()
dma1.sendchannel.start()

dma1.sendchannel.transfer(in_r)
dma1.recvchannel.transfer(out_r)

print("test")
dma1.sendchannel.wait()
print("test1")

dma1.recvchannel.wait()


test
test1


In [23]:
print(out_r)

[ 0 28 26  6 22 10 12 16 14 18 20  8 24  4  2 30 25  5  3 31 15 19 21  9 23
 11 13 17  1 29 27  7 21  9 15 19  3 31 25  5 27  7  1 29 13 17 23 11 12 16
 22 10 26  6  0 28  2 30 24  4 20  8 14 18 13 17 23 11 27  7  1 29  3 31 25
  5 21  9 15 19 20  8 14 18  2 30 24  4 26  6  0 28 12 16 22 10 24  4  2 30
 14 18 20  8 22 10 12 16  0 28 26  6  1 29 27  7 23 11 13 17 15 19 21  9 25
  5  3 31 19 15  9 21  5 25 31  3 29  1  7 27 11 23 17 13 10 22 16 12 28  0
  6 26  4 24 30  2 18 14  8 20  6 26 28  0 16 12 10 22  8 20 18 14 30  2  4
 24 31  3  5 25  9 21 19 15 17 13 11 23  7 27 29  1 30  2  4 24  8 20 18 14
 16 12 10 22  6 26 28  0  7 27 29  1 17 13 11 23  9 21 19 15 31  3  5 25 11
 23 17 13 29  1  7 27  5 25 31  3 19 15  9 21 18 14  8 20  4 24 30  2 28  0
  6 26 10 22 16 12]


In [3]:
 saved = [ 0 ,
 28 ,
 26 ,
  6 ,
 22 ,
 10 ,
 12 ,
 16 ,
 14 ,
 18 ,
 20 ,
  8 ,
 24 ,
  4 ,
  2 ,
 30 ,
 25 ,
  5 ,
  3 ,
 31 ,
 15 ,
 19 ,
 21 ,
  9 ,
 23 ,
 11 ,
 13 ,
 17 ,
  1 ,
 29 ,
 27 ,
  7 ,
 21 ,
  9 ,
 15 ,
 19 ,
  3 ,
 31 ,
 25 ,
  5 ,
 27 ,
  7 ,
  1 ,
 29 ,
 13 ,
 17 ,
 23 ,
 11 ,
 12 ,
 16 ,
 22 ,
 10 ,
 26 ,
  6 ,
  0 ,
 28 ,
  2 ,
 30 ,
 24 ,
  4 ,
 20 ,
  8 ,
 14 ,
 18 ,
 13 ,
 17 ,
 23 ,
 11 ,
 27 ,
  7 ,
  1 ,
 29 ,
  3 ,
 31 ,
 25 ,
  5 ,
 21 ,
  9 ,
 15 ,
 19 ,
 20 ,
  8 ,
 14 ,
 18 ,
  2 ,
 30 ,
 24 ,
  4 ,
 26 ,
  6 ,
  0 ,
 28 ,
 12 ,
 16 ,
 22 ,
 10 ,
 24 ,
  4 ,
  2 ,
 30 ,
 14 ,
 18 ,
 20 ,
  8 ,
 22 ,
 10 ,
 12 ,
 16 ,
  0 ,
 28 ,
 26 ,
  6 ,
  1 ,
 29 ,
 27 ,
  7 ,
 23 ,
 11 ,
 13 ,
 17 ,
 15 ,
 19 ,
 21 ,
  9 ,
 25 ,
  5 ,
  3 ,
 31 ,
 19 ,
 15 ,
  9 ,
 21 ,
  5 ,
 25 ,
 31 ,
  3 ,
 29 ,
  1 ,
  7 ,
 27 ,
 11 ,
 23 ,
 17 ,
 13 ,
 10 ,
 22 ,
 16 ,
 12 ,
 28 ,
  0 ,
  6 ,
 26 ,
  4 ,
 24 ,
 30 ,
  2 ,
 18 ,
 14 ,
  8 ,
 20 ,
  6 ,
 26 ,
 28 ,
  0 ,
 16 ,
 12 ,
 10 ,
 22 ,
  8 ,
 20 ,
 18 ,
 14 ,
 30 ,
  2 ,
  4 ,
 24 ,
 31 ,
  3 ,
  5 ,
 25 ,
  9 ,
 21 ,
 19 ,
 15 ,
 17 ,
 13 ,
 11 ,
 23 ,
  7 ,
 27 ,
 29 ,
  1 ,
 30 ,
  2 ,
  4 ,
 24 ,
  8 ,
 20 ,
 18 ,
 14 ,
 16 ,
 12 ,
 10 ,
 22 ,
  6 ,
 26 ,
 28 ,
  0 ,
  7 ,
 27 ,
 29 ,
  1 ,
 17 ,
 13 ,
 11 ,
 23 ,
  9 ,
 21 ,
 19 ,
 15 ,
 31 ,
  3 ,
  5 ,
 25 ,
 11 ,
 23 ,
 17 ,
 13 ,
 29 ,
  1 ,
  7 ,
 27 ,
  5 ,
 25 ,
 31 ,
  3 ,
 19 ,
 15 ,
  9 ,
 21 ,
 18 ,
 14 ,
  8 ,
 20 ,
  4 ,
 24 ,
 30 ,
  2 ,
 28 ,
  0 ,
  6 ,
 26 ,
 10 ,
 22 ,
 16 ,
 12 ]


In [9]:
flag = 0
for i in range(256):
    if(saved[i] != out_r[i]):
        flag=1
        
if(flag==0):
    print("All values Match")
        
        

All values Match
